In [ ]:
import pickle
from google.colab import drive
import pandas as pd

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title get_sql_tag
def get_sql_query_level(sql_query):
    # Function to classify SQL query and return just the level tag
    def classify_sql_query_level(sql_query):
        sql_query_lower = sql_query.lower()
        level = 6  # Default to OTHER

        if "select" in sql_query_lower and "from" in sql_query_lower:
            level = 0  # SELECT
            if "max(" in sql_query_lower or "min(" in sql_query_lower or \
               "avg(" in sql_query_lower or "count(" in sql_query_lower or \
               "sum(" in sql_query_lower:
                level = 1  # SIMPLE_AGGR

        if "where" in sql_query_lower:
            level = 2  # WHERE

        if "group by" in sql_query_lower:
            level = 3  # GROUPBY

        if "having" in sql_query_lower:
            level = 4  # HAVING

        if "order by" in sql_query_lower:
            level = 5  # ORDERBY

        return level

    # Classification tags
    classification_tags = {
        0: "SELECT",
        1: "SIMPLE_AGGR",
        2: "WHERE",
        3: "GROUPBY",
        4: "HAVING",
        5: "ORDERBY",
        6: "OTHER"
    }

    # Classify the SQL query
    level = classify_sql_query_level(sql_query)

    # Return the classification tag
    return classification_tags[level]

In [ ]:
db_save_path_spider = '/content/drive/MyDrive/spider/test_database'


with open('/content/drive/MyDrive/spider/db_table_to_df.pkl', 'rb') as file:
    db_to_df = pickle.load(file)

spider_train_pd = pd.read_pickle('/content/drive/MyDrive/spider/SPIDER_Simple_cleaned_final.pkl')
spider_train_pd=spider_train_pd.drop(columns=["query_toks","query_toks_no_value","question_toks","operator"])
spider_train_pd['sql_tags'] = spider_train_pd['query'].apply(get_sql_query_level)

In [ ]:
with open('/content/drive/MyDrive/ProprietaryDatasetsDict.pkl', 'rb') as f:
      db_to_df= pickle.load(f)

In [ ]:
qatch_pickle_training = pd.read_pickle('/content/drive/MyDrive/MakingRequestsTo_GPT_data/gpt4o-110_examples_temp_0.pkl')
qatch_pickle = pd.read_pickle('/content/drive/MyDrive/spider/QATCH_proprietary_v2.pkl')

In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
ProprietaryDomains=['medicine','miscellaneous','ecommerce','finance']
qatch_pickle=qatch_pickle[qatch_pickle['domain']==ProprietaryDomains[0]]
qatch_pickle['sql_tags'] = qatch_pickle['sql_tags'].apply(lambda x: x.split('-')[0])## So that we can Having/select Having grouped together and not their subcategories
qatch_pickle_training['sql_tags'] = qatch_pickle_training['sql_tags'].apply(lambda x: x.split('-')[0])## So that we can Having/select Having grouped together and not their subcategories

<ipython-input-7-d7403781a6de>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qatch_pickle['sql_tags'] = qatch_pickle['sql_tags'].apply(lambda x: x.split('-')[0])## So that we can Having/select Having grouped together and not their subcategories


In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
percentage_valid = 0.3
temp_indices = qatch_pickle.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_valid, random_state=42)).index
temp_df = qatch_pickle.loc[temp_indices]
train_df_full = qatch_pickle.drop(temp_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {qatch_pickle.shape[0]}")
print(f"Training set size: {train_df_full.shape[0]}")
print(f"Tempset size: {temp_df.shape[0]}")

Total dataset size: 429
Training set size: 300
Tempset size: 129


In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
percentage_test = 0.5
test_indices = temp_df.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_test, random_state=42)).index
test_df = qatch_pickle.loc[test_indices]
valid_df = temp_df.drop(test_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {temp_df.shape[0]}")
print(f"validation set size: {valid_df.shape[0]}")
print(f"test size: {test_df.shape[0]}")

Total dataset size: 129
validation set size: 66
test size: 63


In [ ]:
# only execute when you want to test/fine tune the model on proprieatry data

# Now lets sample from the trainset as we are not going to use all of it
# At this stage I am still waiting for Simone to change qatch so that I can control
# the amount of queries in each category, and increase the amout of data
train_df_full=qatch_pickle_training.copy()
percentage_train = 1
train_indices = train_df_full.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_train, random_state=42)).index
train_df = train_df_full.loc[train_indices]
# train_df = train_df_full.copy()
# # Check the size of each set to confirm the split
print(f"Total dataset size: {train_df_full.shape[0]}")
print(f"Training set size: {train_df.shape[0]}")


Total dataset size: 560
Training set size: 560


In [ ]:
formatted_tables = {}
for key, df in db_to_df.items():
    # Convert DataFrame to list of rows, keep each row as a separate sublist
    df=df.astype(str)
    rows = df.values.tolist()
    # Get header from DataFrame columns
    header = df.columns.tolist()
    # Creating dictionary entry
    formatted_tables[key] = {'header': header, 'rows': rows}

In [ ]:
# Apply the dictionary to the DataFrame
valid_df['table'] = valid_df['table_used'].apply(lambda x: formatted_tables[x])
valid_df['id']=valid_df['ID']
valid_df['answers'] = valid_df['answer_text'].apply(lambda lst: [str(item) for item in lst])
valid_df = valid_df[['id', 'question', 'answers', 'table']]

In [ ]:
# Apply the dictionary to the DataFrame
train_df['table'] = train_df['table_used'].apply(lambda x: formatted_tables[x])
train_df['id']=train_df['ID']
train_df['answers'] = train_df['answer_text'].apply(lambda lst: [str(item) for item in lst])
train_df = train_df[['id', 'question', 'answers', 'table']]

In [ ]:
train_df.to_json('/content/drive/MyDrive/TAPEX_Ressources/QATCH_DATA_TAPEX/LLM_variations_medicine_100prct.json', orient='records', lines=True)

In [ ]:
valid_df.to_json('/content/drive/MyDrive/TAPEX_Ressources/QATCH_DATA_TAPEX/LLM_variations_medicine_valid.json', orient='records', lines=True)

### Needed format for tapex :
['id', 'question', 'answers', 'table']

In [ ]:
formatted_tables = {}
for key, df in db_to_df.items():
    # Convert DataFrame to list of rows, keep each row as a separate sublist
    df=df.astype(str)
    rows = df.values.tolist()
    # Get header from DataFrame columns
    header = df.columns.tolist()
    # Creating dictionary entry
    formatted_tables[key] = {'header': header, 'rows': rows}

In [ ]:
# Apply the dictionary to the DataFrame
spider_train_pd['table'] = spider_train_pd['seq_id'].apply(lambda x: formatted_tables[x])
spider_train_pd['id']=spider_train_pd['ID']
spider_train_pd['answers'] = spider_train_pd['answer_text'].apply(lambda lst: [str(item) for item in lst])

In [ ]:
# Drop all other columns besides these: ['id', 'question', 'answers', 'table']
spider_train_pd = spider_train_pd[['id', 'question', 'answers', 'table','sql_tags']]

In [ ]:
##  train/valid split for Spider trainset
percentage_valid = 0.2
valid_indices = spider_train_pd.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_valid, random_state=42)).index
valid_df = spider_train_pd.loc[valid_indices]
train_df = spider_train_pd.drop(valid_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {spider_train_pd.shape[0]}")
print(f"Training set size: {train_df.shape[0]}")
print(f"Validation set size: {valid_df.shape[0]}")

Total dataset size: 1441
Training set size: 1154
Validation set size: 287


In [ ]:
# train_df=train_df.drop(['sql_tags'], axis=1)
valid_df=valid_df.drop(['sql_tags'], axis=1)

In [ ]:
# Save the updated DataFrames to JSON files
# train_df.to_json('/content/drive/MyDrive/TAPEX_Ressources/QATCH_DATA_TAPEX/train_df_spider_0prct.json', orient='records', lines=True)
valid_df.to_json('/content/drive/MyDrive/TAPEX_Ressources/QATCH_DATA_TAPEX/valid_df_spider_10prct.json', orient='records', lines=True)

In [ ]:
# Load Data
qatch_pickle = pd.read_pickle('/content/drive/MyDrive/spider/QATCH_SPIDER.pkl')

# Calculate Sample Size
percentage = 1  # Use 100% of data for training in this case
number_of_example = int(percentage * train_df.shape[0])
number_of_cat = qatch_pickle['sql_tags'].nunique()
# Stratified Sampling (if needed)

qatch_sampled = qatch_pickle.groupby('sql_tags').apply(lambda x: x.sample(n=int(number_of_example/number_of_cat),replace=True)).reset_index(drop=True)


# Drop and Reset Index (only if necessary)
# train_df_sampled = train_df_sampled.drop(columns=['ID'])
qatch_sampled = qatch_sampled.reset_index().rename(columns={'index': 'ID'})


In [ ]:
formatted_tables = {}
for key, df in db_to_df.items():
    # Convert DataFrame to list of rows, keep each row as a separate sublist
    df=df.astype(str)
    rows = df.values.tolist()
    # Get header from DataFrame columns
    header = df.columns.tolist()
    # Creating dictionary entry
    formatted_tables[key] = {'header': header, 'rows': rows}

In [ ]:
# Apply the dictionary to the DataFrame
qatch_sampled['table'] = qatch_sampled['seq_id'].apply(lambda x: formatted_tables[x])
qatch_sampled['answers'] = qatch_sampled['answer_text'].apply(lambda lst: [str(item) for item in lst])

In [ ]:
# Drop all other columns besides these: ['id', 'question', 'answers', 'table']
qatch_sampled = qatch_sampled[['question', 'answers', 'table']]

In [ ]:
train_df_copy=train_df[['question', 'answers', 'table']]

In [ ]:
result = pd.concat([qatch_sampled, train_df_copy])
result = result.reset_index().rename(columns={'index': 'id'})

In [ ]:
result

,id,question,answers,table
0,0,"Show the different `Headquarters`, `Main_Indus...","[USA, Oil and gas, USA, Conglomerate, Netherla...","{'header': ['Company_ID', 'Rank', 'Company', '..."
1,1,"Show the different `City`, `membership_amount`...","[London, 112, London, 11, London, 6, Oxford, 7...","{'header': ['Branch_ID', 'Name', 'Open_year', ..."
2,2,"Show the different `Role` in the table ""member""","[Prime Violin, Violin, Viola, Cello, Bass]","{'header': ['Member_ID', 'Name', 'Nationality'..."
3,3,Show the different `Num_of_staff` in the table...,"[13, 19, 42, 27, 24, 34, 425, 28, 23, 16]","{'header': ['Shop_ID', 'Address', 'Num_of_staf..."
4,4,"Show the different `Year`, `Branch_ID` in the ...","[2018, 3, 2018, 2, 2018, 5, 2017, 1, 2018, 1, ...","{'header': ['Member_ID', 'Branch_ID', 'Year', ..."
...,...,...,...,...
2301,1689,"What are the titles, years, and directors of a...","[Jill Rips, 2000, Anthony Hickox, Storm Catche...","{'header': ['movie_id', 'Title', 'Year', 'Dire..."
2302,1690,How many movie directors are there?,[9],"{'header': ['movie_id', 'Title', 'Year', 'Dire..."
2303,1691,Count the number of different directors.,[9],"{'header': ['movie_id', 'Title', 'Year', 'Dire..."
2304,1692,Show all director names who have a movie in th...,"[Troy Duffy, John Swanbeck, Anthony Hickox, An...","{'header': ['movie_id', 'Title', 'Year', 'Dire..."


In [ ]:
result.to_json('/content/drive/MyDrive/TAPEX_Ressources/QATCH_DATA_TAPEX/train_df_spider_100prct.json', orient='records', lines=True)